In [ ]:
#r "nuget: GraphQL.Client"
#r "nuget: GraphQL.Client.Serializer.Newtonsoft"
#r "nuget: Plotly.NET.Interactive,  2.0.0"

open GraphQL
open GraphQL.Client.Http
open GraphQL.Client.Serializer.Newtonsoft
open System
open Plotly.NET

let graphQLClient = new GraphQLHttpClient("https://camaradosdeputadosgql-qboe6n5gda-uc.a.run.app/graphql", new NewtonsoftJsonSerializer());

let deputyRequest party state year = new GraphQLRequest (
    $"""
    query {{
        deputies (party: "{party}", state: "{state}") {{
            name
            party
            expenses (year: "{year}") {{
                netValue
                documentDate
            }}
        }}
    }}
    """
)

let expenseRequest name year = new GraphQLRequest (
    $"""
    query {{
        deputies (name: "{name}") {{
            name
            party
            state
            expenses (year: "{year}") {{
                netValue
                documentDate
                supplierCnpjOrCpf
                supplierName
            }}
        }}
    }}
    """
)

type DeputyResponse = {
    deputies: Deputy list
} 
and Deputy = {
    name: string
    party: string
    expenses: Expense[]
}
and Expense = {
    netValue: decimal
    supplierName: string
    documentDate: Nullable<DateTime>
}

let GetDeputies party state year =
    let response =
        graphQLClient.SendQueryAsync<DeputyResponse>(deputyRequest party state year)
        |> Async.AwaitTask
        |> Async.RunSynchronously
    response.Data.deputies

let GetExpenses deputy year = 
    let response = 
        graphQLClient.SendQueryAsync<DeputyResponse>(expenseRequest deputy year)
        |> Async.AwaitTask
        |> Async.RunSynchronously
    response.Data.deputies |> Seq.head

let scatter (deputy:Deputy) =
    let dates, values =
        if deputy.expenses <> null then
            deputy.expenses
            |> Array.filter(fun expense -> expense.documentDate.HasValue)
            |> Array.sortBy(fun expense -> expense.documentDate.Value)
            |> Array.map (fun expense -> expense.documentDate.Value, expense.netValue)
            |> Array.unzip
        else
            [||],[||]
    Chart.Line(dates, values)
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    |> Chart.withTraceInfo deputy.name

let expenseChart (deputy:Deputy) =
    let labels, values =
        deputy.expenses
        |> Seq.groupBy(fun expense -> expense.supplierName)
        |> Seq.map (fun (name, expenses) ->
            let total =
                expenses
                |> Seq.sumBy ( fun expense -> expense.netValue )
            name, total
        )
        |> Seq.toList
        |> List.unzip
    Chart.Pie(Labels=labels, values=values)
    |> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
    

Installed Packages GraphQL.Client, 5.1.0 GraphQL.Client.Serializer.Newtonsoft, 5.1.0 Plotly.NET.Interactive, 2.0.0

In [ ]:
GetDeputies "PT" "SP" "2021"
|> Seq.map scatter
|> Chart.combine
|> Chart.withXAxisStyle("Data")

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
GetExpenses "Ivan Valente" 2022
|> expenseChart
|> Chart.withTitle "Despesas por fornecedor - Ivan Valente - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
GetExpenses "Luiza Erundina" 2022
|> expenseChart
|> Chart.withTitle "Despesas por fornecedor - Luiza Erundina - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
GetExpenses "Eduardo Bolsonaro" 2022
|> expenseChart
|> Chart.withTitle "Despesas por fornecedor - Eduardo Bolsonaro - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
GetExpenses "Alexandre Frota" 2022
|> expenseChart
|> Chart.withTitle "Despesas por fornecedor - Alexandre Frota - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
GetExpenses "Pr. Marco Feliciano" 2022
|> expenseChart
|> Chart.withTitle "Despesas por fornecedor - Pr. Marco Feliciano - 2022"

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div